### What is an Embedding?

An embedding is a way to convert text into numbers (vectors) so that a machine can understand meaning.

Text  →  Embedding Model  →  Vector (numbers)


1. Vectors: Embeddings are arrays of numbers. Each text → one vector. Example: [0.23, -0.11, 0.78, ...]
2. Dimension: Length of the vector. More dimensions → better meaning capture (usually). Examples: 384 dimensions, 768 dimensions
3. Semantic Similarity: Similar meaning → vectors closer together
4. Distance / Similarity Measures: Used to compare vectors:
    1. Cosine similarity (most common): Higher similarity score = closer meaning
    2. Euclidean distance
    3. Dot product

"Silk saree"	"Banarasi saree"	Very similar
"Silk saree"	"Laptop charger"	Not similar

### OpenAI Embeddings
What are OpenAI Embeddings?

Pre-trained, high-quality embeddings provided by OpenAI.

Popular model:

+ text-embedding-3-small
+ text-embedding-3-large

In [1]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv() 

emb = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vector = emb.embed_query("What is a Information technology?")
#print(vector)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector = emb.embed_query("What is a Information technology?")
print(len(vector))


384


### What is a Vector Database?

A vector database stores embeddings (vectors) and allows fast similarity search.

Situations where this is used:
+ Data is unstructured (text, PDFs, images)
+ We want meaning-based search, not keywords

Text → Embedding → Vector DB → Similarity Search

### Why Vector Databases Are Needed

1. Normal DBs
      1. Store rows & columns
      2. Cannot compare meaning
2. Vector DBs
      1. Store vectors
      2. Find closest meaning

Some of the applications of Vector Database:
+ RAG systems
+ Chatbots
+ Semantic search
+ Recommendation systems

Metadata
Extra info stored with vectors:
+ filename
+ category
+ page number
+ source

Used for filtering results

### FAISS (Facebook AI Similarity Search)
What is FAISS?

+ Library by Meta
+ Runs locally
+ Best for learning & prototyping

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

db = FAISS.from_texts(
    ["Silk saree", "Cotton saree"],
    OpenAIEmbeddings()
)

### Chroma
What is Chroma?

+ Developer-friendly vector DB
+ Supports local + persistent storage

In [21]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_texts(
    ["Silk saree", "Cotton saree"],
    embedding=OpenAIEmbeddings(),
    persist_directory="./chroma_db"
)


### Pinecone
What is Pinecone?

+ Managed cloud vector database
+ Production-ready

### What is Vector Store Indexing? (1-line idea)

Vector store indexing is how we store text as vectors along with extra info, so we can search, filter, and retrieve the most relevant content efficiently.

Why metadata matters?

+ Helps filter results
+ Adds context
+ Makes retrieval more accurate

##### What is filtering?
Retrieving vectors only if metadata conditions match.

##### Why filtering is important?
+ Avoids irrelevant data
+ Improves answer quality
+ Faster searches

##### What is similarity search?

Finding vectors closest in meaning, not exact words.

How it works
Query → embedding

Compare with stored embeddings

Return top-k closest vectors

#### MMR Search (Max Marginal Relevance)
Problem with normal similarity search
+ Results may be too similar to each other
+ Less information coverage

##### What MMR does

Balances:

+ Relevance
+ Diversity

https://reference.langchain.com/python/langchain_core/vectorstores/#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search

Use MMR when:

+ FAQs
+ Knowledge bases
+ Educational content
+ Product recommendations

Intuition Behind MMR (Human Thinking)

Think like a teacher 👩‍🏫:

“I want good answers, but don’t repeat the same idea again and again.”

MMR thinks:

“This result is relevant”

“But is it too similar to what I already selected?”

+ If yes >>>>> skip
+ If no >>>>> include

MMR Steps:

1. Convert query >>>>> embedding
2. Fetch top fetch_k similar docs
3. Select the most relevant document first
4. For each next selection:
     + Penalize docs similar to already selected ones
5. Repeat until k docs selected

Interview Content
+ MMR reduces redundancy in vector search
+ MMR balances relevance and diversity
+ MMR is ideal for exploratory queries
+ Lambda controls relevance vs diversity
+ MMR improves RAG context quality

In [113]:
# pip install -U langchain-core langchain-community faiss-cpu

from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.embeddings import FakeEmbeddings

# 1) Sample docs (intentionally overlapping topics)
docs = [
    Document(page_content="Silk sarees are best for weddings and festivals.", metadata={"type": "saree"}),
    Document(page_content="Pure silk sarees are traditional and premium.", metadata={"type": "saree"}),
    Document(page_content="Cotton sarees are light and great for summer daily wear.", metadata={"type": "saree"}),
    Document(page_content="Linen sarees are breathable and modern-looking.", metadata={"type": "saree"}),
    Document(page_content="Returns are accepted within 7 days if unused.", metadata={"type": "policy"}),
    Document(page_content="Delivery takes 5 to 7 business days.", metadata={"type": "policy"}),
    Document(page_content="Payments: UPI, credit card, debit card, net banking.", metadata={"type": "payment"}),
]

# 2) Build vector store using FakeEmbeddings (no network, no key)
emb = FakeEmbeddings(size=1536)
vs = FAISS.from_documents(docs, emb)

query = "Tell me about sarees"

# 3) Regular similarity search (often repetitive)
sim = vs.similarity_search(query, k=4)

print("=== Similarity Search ===")
for i, d in enumerate(sim, 1):
    print(f"\n{i}. {d.page_content}")

# 4) MMR search (relevant + diverse)
mmr = vs.max_marginal_relevance_search(
    query=query,
    k=4,
    fetch_k=10,
    lambda_mult=0.5
)

print("\n\n=== MMR Search ===")
for i, d in enumerate(mmr, 1):
    print(f"\n{i}. {d.page_content}")


=== Similarity Search ===

1. Returns are accepted within 7 days if unused.

2. Silk sarees are best for weddings and festivals.

3. Pure silk sarees are traditional and premium.

4. Payments: UPI, credit card, debit card, net banking.


=== MMR Search ===

1. Returns are accepted within 7 days if unused.

2. Delivery takes 5 to 7 business days.

3. Pure silk sarees are traditional and premium.

4. Linen sarees are breathable and modern-looking.


### What is Chunking? (Core Definition)

Chunking is the process of splitting large text into smaller, meaningful pieces (chunks) before converting them into embeddings.

##### What happens without chunking?
+ Token limit exceeded because LLMs have max token size
+ Poor retrieval happens as Embedding mixes unrelated topics
+ Wrong answers are displayed because of Irrelevant context retrieved
+ Higher cost and Larger input will eventually lead to higher cost

A good chunk should:
+ Contain one main idea
+ Be self-contained
+ Not be too small or too large
+ Fit within embedding model limits

#### Chunk Size (Most Important Decision)
What is chunk size?

Number of tokens or characters per chunk.

##### Common chunk sizes
Use case	Chunk size
+ Short FAQ	200–400
+ Blogs / docs	500–800
+ PDFs / manuals	800–1000
+ Code	150–300

##### Chunk Overlap (Why It Exists)
What is overlap?

Repeating some part of previous chunk in the next chunk.

Why overlap is needed

+ Prevents sentence cut-off
+ Preserves context
+ Improves recall

1. Chunk 1: tokens 0–500
2. Chunk 2: tokens 400–900
3. Overlap = 100 tokens

Fixed-Size Chunking (Baseline Strategy)

How it works: Split text purely by size.

    Document(page_content="Silk sarees are best for weddings and festivals.", metadata={"type": "saree"}),
    Document(page_content="for weddings and festivals. Pure silk sarees are traditional and premium.", metadata={"type": "saree"}),

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_text(text)

##### Recursive Chunking (Recommended)
How it works

Splits text by:

Paragraph

Sentence

Word

Character (fallback)

This preserves meaning better

#### Semantic Chunking (Advanced)

Split text based on meaning, not size.

Following are Techniques

+ Headings based split
+ Paragraph based split
+ Topic change detection
+ LLM-assisted splitting

Best suited for:
+ Legal documents
+ Research papers
+ Educational content

> Slower and costlier

#### Chunking with Metadata (Best Practice)

Always attach metadata to each chunk.

Interview One-Liners

+ Chunking controls retrieval quality
+ Chunk size affects cost and accuracy
+ Overlap preserves context
+ Semantic chunking > fixed chunking
+ Poor chunking breaks RAG

In [74]:
text = """
Amazon is a premium online brand that sells silk, cotton, and linen sarees.
Silk sarees are best suited for weddings and festivals.
Cotton sarees are lightweight and ideal for daily wear and summer.
Linen sarees are breathable and modern.

Delivery usually takes 5 to 7 business days.
Returns are accepted within 7 days if the product is unused.
Refunds are processed within 3 business days after pickup.

Payment methods include UPI, credit cards, debit cards, and net banking.
Cash on delivery is available for select locations.
"""


In [76]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

embeddings = OpenAIEmbeddings()

doc = Document(page_content=text)
vectorstore_no_chunk = FAISS.from_documents([doc], embeddings)


In [80]:
results = vectorstore_no_chunk.similarity_search(
    "What are the delivery timelines?",
    k=1
)

# Answers without chunking 
print(results[0].page_content)


Amazon is a premium online brand that sells silk, cotton, and linen sarees.
Silk sarees are best suited for weddings and festivals.
Cotton sarees are lightweight and ideal for daily wear and summer.
Linen sarees are breathable and modern.

Delivery usually takes 5 to 7 business days.
Returns are accepted within 7 days if the product is unused.
Refunds are processed within 3 business days after pickup.

Payment methods include UPI, credit cards, debit cards, and net banking.
Cash on delivery is available for select locations.



In [84]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = splitter.split_text(text)

print(f"Total chunks created: {len(chunks)}")
for i, c in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ---\n{c}")


Total chunks created: 2

--- Chunk 1 ---
Amazon is a premium online brand that sells silk, cotton, and linen sarees.
Silk sarees are best suited for weddings and festivals.
Cotton sarees are lightweight and ideal for daily wear and summer.
Linen sarees are breathable and modern.

--- Chunk 2 ---
Delivery usually takes 5 to 7 business days.
Returns are accepted within 7 days if the product is unused.
Refunds are processed within 3 business days after pickup.

Payment methods include UPI, credit cards, debit cards, and net banking.
Cash on delivery is available for select locations.


In [92]:
documents = [Document(page_content=chunk) for chunk in chunks]

vectorstore_chunked = FAISS.from_documents(documents, embeddings)

In [94]:
results = vectorstore_chunked.similarity_search(
    "What is the delivery time?",
    k=2
)

for r in results:
    print("\n--- Retrieved Chunk ---")
    print(r.page_content)


--- Retrieved Chunk ---
Delivery usually takes 5 to 7 business days.
Returns are accepted within 7 days if the product is unused.
Refunds are processed within 3 business days after pickup.

Payment methods include UPI, credit cards, debit cards, and net banking.
Cash on delivery is available for select locations.

--- Retrieved Chunk ---
Amazon is a premium online brand that sells silk, cotton, and linen sarees.
Silk sarees are best suited for weddings and festivals.
Cotton sarees are lightweight and ideal for daily wear and summer.
Linen sarees are breathable and modern.


In [99]:
# Smaller chunks 

small_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=30
)

small_chunks = small_splitter.split_text(text)
print(len(small_chunks))

5


In [101]:
large_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)

large_chunks = large_splitter.split_text(text)
print(len(large_chunks))


1


In [105]:
no_overlap_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=0
)

no_overlap_chunks = no_overlap_splitter.split_text(text)

# Overlap preserves continuity

In [107]:
documents = []

for i, chunk in enumerate(chunks):
    documents.append(
        Document(
            page_content=chunk,
            metadata={
                "source": "mycompany_policy",
                "chunk_id": i
            }
        )
    )


In [109]:
results = vectorstore_chunked.max_marginal_relevance_search(
    query="Tell me about sarees",
    k=3,
    fetch_k=6,
    lambda_mult=0.5
)

for r in results:
    print("\n--- MMR Chunk ---")
    print(r.page_content)



--- MMR Chunk ---
Amazon is a premium online brand that sells silk, cotton, and linen sarees.
Silk sarees are best suited for weddings and festivals.
Cotton sarees are lightweight and ideal for daily wear and summer.
Linen sarees are breathable and modern.

--- MMR Chunk ---
Delivery usually takes 5 to 7 business days.
Returns are accepted within 7 days if the product is unused.
Refunds are processed within 3 business days after pickup.

Payment methods include UPI, credit cards, debit cards, and net banking.
Cash on delivery is available for select locations.
